We are going to create a dataset where each observation has a company, a date, that company's trailing stock returns and volatility over different lengths, as well as that information for the market benchmark (SP500). To do so, we will use a Compustat dataset of monthly stock prices for companies on American exchanges and a Yahoo Finance dataset of SP500 monthly prices.

__________________________________________________________________________________________________________________________


We begin by importing IntegrateData subclasses responsible for creating and merging the datasets. 

IntegrateData is an abstract superclass which all integrators inherit from. It defines methods for reading and writing csv data either locally or on S3, as well as maintaining a copy of such data in a pandas dataframe. Subclasses must define the integrate_data() method, which executes the necessary manipulations to the internally stored data such that it is ready for output. For example, the IntegrateStockData subclass' integrate_data() method uses price information in its internal df to calculate and write new columns with hold historical stock returns.

The only IntegrateData subclass method that is publically called is process(), which runs the sequence of reading, processing, cleaning, and writing the data as specified by the constructor and the subclass' concrete implementation of integrate_data().

In [2]:
from integrate_stock import IntegrateStockData
from volatility_data import IntegrateVolatilityData
from sp500_data import IntegrateSPData
from sp500_data_merge import MergeSPWithMaster

The first dataset to construct will have stock returns and volatility. We import a file named 'na-monthly-share-price.csv', currently stored in S3. The IntegrateStockData object will calculate returns based on prices, add that data to the existing data, and write that to the location specified by stock_returns_path.

In [3]:
stock_price_path = 'na-monthly-share-price.csv'
stock_returns_path = 'na-monthly-share-returns.csv'

integrate_price = IntegrateStockData(stock_price_path, stock_returns_path, input_type='s3')
integrate_price.process()

Integrating 36 month trailing returns: 100%|██████████| 2749846/2749846 [03:55<00:00, 11691.60it/s]


Okay.. 20 minutes isn't too bad for 5 returns periods each with 2.7 million observations.

Next, we add volatility data to the mix. This is done through an IntegrateVolatilityData object, which uses the pandas pd.DataFrame.std() method to calculate standard deviations of trailing returns. Given that each data point now requires accessing and performing operations on an array of data points, rather than two singletons, this takes a bit (lot) longer than calculating returns. 

In [7]:
stock_returns_vol_path = 'na-monthly-share-returns.csv'

integrate_volatility = IntegrateVolatilityData(stock_returns_path, stock_returns_vol_path)
integrate_volatility.process()

Integrating 12 month historical volatility: 100%|██████████| 2749846/2749846 [14:23<00:00, 3183.93it/s]


45 minutes.. just enough time to debug another DateTime error. Our stock dataset now has all of the necessary individual performance data, so it's time to create the SP500 performance dataset. The IntegrateSPData class' integrate_data() method essentially replicates the functionality of the IntegrateStockData and IntegrateVolatilityData classes, adjusted for the Yahoo Finance dataset formatting.

In [3]:
sp_500_price_path = 'sp500-monthly-price.csv'
sp_500_returns_vol_path = 'sp500-returns-volatility.csv'

construct_sp_data = IntegrateSPData(sp_500_price_path, sp_500_returns_vol_path, input_type='s3')
construct_sp_data.process()

Integrating SP500 12 month historical volatility: 100%|██████████| 243/243 [00:00<00:00, 3087.05it/s]


Nice, that one was fast. The only thing left to do is merge the two datasets! The MergeSPWithMaster class seems to be a good pick for that. It merges by matching the dates in the stock and SP500 datasets, so that each observation for a stock has added market benchmark information.

In [5]:
master_data_path = '1_performance_data.csv'

merge_data = MergeSPWithMaster(stock_returns_vol_path, sp_500_returns_vol_path, master_data_path, output_type='s3')
merge_data.process()

Merging stock performance and market performance datasets: 100%|██████████| 42/42 [00:09<00:00,  4.57it/s]


Data has now been merged and written to S3! All in a day's work.